In [1]:
# 引入文档
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
# 格式化数据
import pandas as pd

df = pd.DataFrame(documents, columns=['course', 'section', 'question', 'text'])
df.head()

,course,section,question,text
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
2,data-engineering-zoomcamp,General course-related questions,Course - Can I still join the course after the...,"Yes, even if you don't register, you're still ..."
3,data-engineering-zoomcamp,General course-related questions,Course - I have registered for the Data Engine...,You don't need it. You're accepted. You can al...
4,data-engineering-zoomcamp,General course-related questions,Course - What can I do before the course starts?,You can start by installing and setting up all...


In [3]:
# 批量处理所有文档
from sklearn.feature_extraction.text import TfidfVectorizer
fields = ['section', 'question', 'text']
transformers = {}
matrices = {}

for field in fields:
    cv = TfidfVectorizer(stop_words='english', min_df=3)
    X = cv.fit_transform(df[field])

    transformers[field] = cv
    matrices[field] = X

transformers['text'].get_feature_names_out()
matrices['text']

<948x2118 sparse matrix of type '<class 'numpy.float64'>'
	with 26463 stored elements in Compressed Sparse Row format>

In [4]:
# 通过SVD将文档进行嵌入
from sklearn.decomposition import TruncatedSVD

X = matrices['text']
cv = transformers['text']

X

<948x2118 sparse matrix of type '<class 'numpy.float64'>'
	with 26463 stored elements in Compressed Sparse Row format>

In [10]:
# 使用SVD，它会将文档的原始内容尽可能保留并降维
svd = TruncatedSVD(n_components=16)
X_emb = svd.fit_transform(X)

X_emb.shape

(948, 16)

In [11]:
X_emb[0]

array([ 0.08800073, -0.07522612, -0.1005397 ,  0.05179391,  0.05356622,
       -0.0584842 ,  0.02909418,  0.06024781, -0.19378915,  0.33482855,
        0.06852973,  0.06651945, -0.1227339 ,  0.04540146, -0.01822307,
       -0.02478234])

In [12]:
# 对询问语句可以做相同的操作
query = 'I just singned up. Is it too late to join the course?'

Q = cv.transform([query])
Q_emb = svd.transform(Q)
Q_emb[0]

array([ 0.04353668, -0.03077025, -0.04418478,  0.01177329,  0.02646575,
       -0.05036082,  0.01396342,  0.03265892, -0.11419632,  0.18320822,
        0.0608642 ,  0.04739668, -0.07505804,  0.02486528,  0.01532271,
       -0.04092136])

In [14]:
import numpy as np
np.dot(X_emb[0], Q_emb[0])

0.11980810517411097

In [17]:
# 计算其与所有文档的相似度
from sklearn.metrics.pairwise import cosine_similarity
score = cosine_similarity(X_emb, Q_emb).flatten()
score

array([ 9.78256245e-01,  7.71812597e-02,  9.56596337e-01,  9.30028785e-01,
        7.83846077e-02,  6.61092968e-01,  7.07181760e-01,  9.80211537e-01,
        8.60445342e-01,  6.27741806e-01,  7.78843720e-01,  9.81555643e-01,
        8.84521167e-01,  9.47962886e-01,  2.58448009e-01,  9.77384629e-01,
        5.03226799e-01,  7.29989656e-01,  7.50369142e-01,  4.40227133e-01,
        5.82800464e-01,  9.35727285e-03,  8.71609589e-01,  7.32743484e-01,
        1.38601831e-01,  1.66024696e-01,  3.40344059e-01,  8.85387260e-01,
        7.08582665e-01,  7.67479906e-01,  3.69294835e-01,  4.43073893e-01,
        7.98002893e-01,  6.52822393e-01,  1.70848750e-01,  8.56348751e-01,
        4.01361088e-01,  1.86118516e-01,  6.57513872e-01,  4.80062275e-01,
        5.68822026e-01,  2.13439959e-01,  3.39054455e-01,  1.55594278e-01,
        1.53597909e-02,  2.37749608e-02, -2.09309279e-01,  1.11678403e-02,
        1.37964244e-02,  1.82335910e-01,  1.70104686e-01, -8.35353177e-02,
        3.28563836e-01, -

In [18]:
idx = np.argsort(-score)[:10]
list(df.loc[idx].text)

['If you have submitted two projects (and peer-reviewed at least 3 course-mates’ projects for each submission), you will get the certificate for the course. According to the course coordinator, Alexey Grigorev, only two projects are needed to get the course certificate.\n(optional) David Odimegwu',
 'The course videos are pre-recorded, you can start watching the course right now.\nWe will also occasionally have office hours - live sessions where we will answer your questions. The office hours sessions are recorded too.\nYou can see the office hours as well as the pre-recorded course videos in the course playlist on YouTube.',
 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a cer

In [20]:
# 同理使用NMF
from sklearn.decomposition import NMF
nmf = NMF(n_components=16)
X_emb = nmf.fit_transform(X)
X_emb[0]

Q = cv.transform([query])
Q_emb = nmf.transform(Q)
Q_emb[0]

score = cosine_similarity(X_emb, Q_emb).flatten()
idx = np.argsort(-score)[:10]
list(df.loc[idx].text)

["The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'Please choose the closest one to your answer. Also do not post your answer in the course slack channel.',
 'No, it’s not possible. The form is closed after the due date. But don’t worry, homework is not mandatory for finishing the course.',
 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of Nov

In [22]:
# 使用Bert模型
import torch
from transformers import BertModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
model.eval()  # Set the model to evaluation mode if not training

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [24]:
# 将文本编码
texts = [
    "Yes, we will keep all the materials after the course finishes.",
    "You can follow the course at your own pace after it finishes"
]
encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
encoded_input

{'input_ids': tensor([[  101,  2748,  1010,  2057,  2097,  2562,  2035,  1996,  4475,  2044,
          1996,  2607, 12321,  1012,   102],
        [  101,  2017,  2064,  3582,  1996,  2607,  2012,  2115,  2219,  6393,
          2044,  2009, 12321,   102,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])}

In [27]:
# 禁用梯度计算进行推理
with torch.no_grad():  # Disable gradient calculation for inference
    outputs = model(**encoded_input)
    hidden_states = outputs.last_hidden_state

In [29]:
hidden_states.shape

torch.Size([2, 15, 768])

In [30]:
# 压缩嵌入
sentence_embeddings = hidden_states.mean(dim=1)
sentence_embeddings.shape

torch.Size([2, 768])

In [31]:
X_emb = sentence_embeddings.numpy()

In [32]:
# 使用cpu试用即可
sentence_embeddings_cpu = sentence_embeddings.cpu()

In [33]:
# 定义函数用于创建batch
def make_batches(seq, n):
    result = []
    for i in range(0, len(seq), n):
        batch = seq[i:i+n]
        result.append(batch)
    return result

In [36]:
# 将文本批处理，通过BERT计算其嵌入向量，并将嵌入组成一个数组
from tqdm import *
texts = df['text'].tolist()
text_batches = make_batches(texts, 8)

all_embeddings = []

for batch in tqdm(text_batches):
    encoded_input = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')

    with torch.no_grad():
        outputs = model(**encoded_input)
        hidden_states = outputs.last_hidden_state
        
        batch_embeddings = hidden_states.mean(dim=1)
        batch_embeddings_np = batch_embeddings.cpu().numpy()
        all_embeddings.append(batch_embeddings_np)

final_embeddings = np.vstack(all_embeddings)

100%|████████████████████████████████████████████████████████████████████████████████████| 119/119 [11:18<00:00,  5.70s/it]


In [37]:
final_embeddings

array([[-0.00456303, -0.11667512,  0.6274718 , ..., -0.03659191,
         0.10031679,  0.0292713 ],
       [-0.1423361 , -0.1985392 ,  0.28455415, ..., -0.01139053,
        -0.1539977 ,  0.09535079],
       [ 0.19672246, -0.08461305,  0.28200513, ...,  0.11395867,
        -0.06448027, -0.0128261 ],
       ...,
       [-0.2821744 , -0.33324358,  0.29784983, ..., -0.35042733,
         0.03266054,  0.09537254],
       [-0.42807093, -0.39468756,  0.3094198 , ..., -0.05943285,
        -0.12965176,  0.07887058],
       [-0.16892126, -0.25146285,  0.47843292, ..., -0.18535416,
        -0.1610892 ,  0.27272922]], dtype=float32)